In [ ]:
import sys; sys.path.insert(0, "C:/GIT/python-local/pygeostat_freeze/")
sys.path.insert(0, '../')
import pygeostat as gs
import gglib as gg
import numpy as np
np.set_printoptions(precision=4, suppress=True)
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import os, copy, shutil, sys, glob, time
from mpl_toolkits.axes_grid1 import ImageGrid
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.backends.backend_pdf import PdfPages
from collections import OrderedDict
import numba
from gglib.datamgmt.simpleio import *
%matplotlib inline

In [ ]:
rm.mpl_setup()
def chapterexport(filename, *args, **kwargs):
    rm.thesis_export(filename, chap=6, **kwargs)

# functions

In [ ]:
def cov(x1, x2):
    """ covariance of the 1D arrays x1 and x2"""
    assert(x1.shape[0] == x2.shape[0]), "must pass arrays of similar shape!"
    m1 = x1.mean(axis=0)
    z1 = x1 - m1
    m2 = x2.mean(axis=0)
    z2 = x2 - m2
    return ((z2 * z1).sum(axis=0) - m1*m2) / x1.shape[0]

def corr(x1, x2):
    """ correlation between 1D arrays x1 and x2 """
    return cov(x1, x2) / (x1.std(axis=0) * x2.std(axis=0))

def rankcorr(x1, x2):
    from scipy.stats import pearsonr
    return pearsonr(x1, x2)[0]

def rmse(pred, truth):
    return np.sqrt(np.mean((pred - truth) ** 2))

def mae(pred, truth):
    return np.mean(np.abs(pred - truth))

import matplotlib.colors as colors
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=256):
    "from: https://stackoverflow.com/a/18926541/5545005"
    cmap = plt.get_cmap(cmap)
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

# project imports and datafiles

In [ ]:
from _oilsands_project_defs import *
qf = gg.QuickFormatter(fs=6)

In [ ]:
project = 'oilsands'

In [ ]:
points = gg.PointContainer(f'{project}_maincats.dat')

In [ ]:
names = ['reals_ensbest', 'reals_random', 'modelcats', 'mvclus', 
         'agglom', 'betteragglom', 'nodomains']
titles = ["Category Realizations",
          "Random Domain", "Geological Categories", 
          "Multivariate Clusters", "Spatial Clusters", "Improved Spatial Clusters",
          "No Domains"]
names = OrderedDict([(k, t) for k, t in zip(names, titles)])

In [ ]:
workdir = 'L:/Thesis/Ch6/oilsands_test/'

In [ ]:
nfolds = 5
ifolds = np.arange(nfolds, dtype=int) + 1
nreal = 100

# Import Fold Files

In [ ]:
valdict = {}
for var in variables:
    for name, title in names.items():
        for ifold in ifolds:
            valdict[var, name, ifold] = \
                read_gslib(f'{workdir}{name}_{var}_fold{ifold}.out').values

## scatplts

In [ ]:
qf = gg.QuickFormatter(fs=6, nticks=(6, 6))
for var in variables:
    acaxs = qf.getax(len(names), shape=(3, 3), figsize=(5.8, 6))
    iax = 0
    for (name, title), ax in zip(names.items(), acaxs):
        etypes = np.empty(0)
        truths = np.empty(0)
        for ifold in ifolds:
            thisdata = valdict[var, name, ifold]
            ids = thisdata[:, 0:1]
            xyz = thisdata[:, 1:4]
            truth = thisdata[:, 4:5].flatten()
            reals = thisdata[:, 5:]
            etype = np.mean(reals, axis=1).flatten()
            truths = np.concatenate((truths, truth))
            etypes = np.concatenate((etypes, etype))
        ax = gs.scatxval(etypes, truths, ax=ax, 
                         title=f'{var}, {title}', ms=1, pltstyle='pt5.5')
        qf(ax, fignum=iax, lblxy=(0.01, 1.05), aspect=1, 
           xlim=(0, None), ylim=(0, None))
        iax += 1
    plt.subplots_adjust(hspace=0.35)
    chapterexport(f'oilsands_scatxval_all_{var}.pdf')

In [ ]:
for var in variables:
    errdf = pd.DataFrame(columns=names.values(), 
                         index=['Covariance', 'Correlation', 'RMSE'])
    for name, title in names.items():
        etypes = np.empty(0)
        truths = np.empty(0)
        for ifold in ifolds:
            thisdata = valdict[var, name, ifold]
            ids = thisdata[:, 0:1]
            xyz = thisdata[:, 1:4]
            truth = thisdata[:, 4]
            reals = thisdata[:, 5:]
            etype = np.mean(reals, axis=1).flatten()
            truths = np.concatenate((truths, truth))
            etypes = np.concatenate((etypes, etype))
        errdf.loc['Covariance', title] = cov(etypes, truths)
        errdf.loc['Correlation', title] = corr(etypes, truths)
        errdf.loc['RMSE', title] = rmse(etypes, truths)
    rm.latex_table(errdf.T, index=True, table_width_cm=9, float_prec=2, 
                  caption=(f'{var} K-Fold error statistics by category, oilsands dataset.'), 
                   label=f'os_errorstats_{var.lower()}'
                  )

## accplts

In [ ]:
accpltdict = {}
for var in variables:
    iax = 0
    for (name, title), ax in zip(names.items(), acaxs):
        probs = None
        for ifold in ifolds:
            thisdata = valdict[var, name, ifold]
            xyz = thisdata[:, :4]
            truth = thisdata[:, 4:5].flatten()
            reals = thisdata[:, 5:]
            thisaccsim, sumstats = gs.accsim(truth, reals)
            if probs is None:
                probs = thisaccsim
            else:
                probs['FracIn'] += thisaccsim['FracIn']
        accpltdict[var, name] = [probs['ProbInt'], probs['FracIn'] / nfolds, sumstats]

In [ ]:
statnames = {
    'avgvar': 'U', 
    'mse': 'MSE', 
    'acc': 'Accuracy', 
    'pre': 'Precision', 
    'goo': 'Goodness'
}

In [ ]:
for var in variables:
    acaxs = qf.getax(len(names), shape=(3, 3), figsize=(5.8, 6))
    iax = 0
    for (name, title), ax in zip(names.items(), acaxs):
        probs = accpltdict[var, name]
        ax = gs.accplt(x=probs[0], y=probs[1], ax=ax, stat_blk=False, 
                       title=f'{var}, {title}', pltstyle='pt5.5', ms=2)
        qf(ax, fignum=iax, lblxy=(0.01, 1.05), 
           annot='\n'.join([f'{statnames[k]} = {probs[2][k]:.3f}' 
                            for k in statnames.keys()]), 
           annha='left', annva='top', annxy=(0.01, 0.99),
           aspect=1, nticks='auto',
          )
        iax += 1
    plt.subplots_adjust(hspace=0.35)
    chapterexport(f'oilsands_accplt_all_{var}.pdf')

# histreproduction

In [ ]:
finaldir = 'L:/Thesis/Ch6/oilsands_final/'

In [ ]:
declus = gg.PolyDecluster(points, griddef.convert_to_2d(), dhcol=points.dh)
points['Declustering Weight'] = declus.decluster().values

In [ ]:
for var in variables:
    acaxs = qf.getax(len(names), shape=(3, 3), figsize=(6, 6))
    iax = 0
    for (name, title), ax in zip(names.items(), acaxs):
        allreals = [gs.readfile(f'{finaldir}{name}/Final/mergedbtr_{i}.gsb')[var] for i in range(nreal)]
        allreals = pd.concat(allreals, axis=1).values
        allreals[allreals <= -999] = np.nan
        ax = gg.histpltsim(allreals, points[var], refwts=points['Declustering Weight'], 
                           ax=ax, title=f'{var}, {title}', stat_fs=6)
        qf(ax, fignum=iax, lblxy=(0.01, 1.05), xlabel=var, ylabel='Cumulative Probability')
        iax += 1    
    plt.tight_layout(pad=1.2)
    chapterexport(f'oilsands_histpltsim_{var}.pdf')
    import gc; gc.collect()

# vario reproduction

In [ ]:
varsim = gs.Program('../exes/varsim')
varsimpar = """START OF PARAMETERS:
    nodata               -file with lithology information
    0   7                        -   lithology column (0=not used), code
    {datfl}                   -file with data
    {nvar}   {varcols}            -   number of variables, column numbers
    {ltrim}     1.0e21    -   trimming limits
    {outfl}      -output file for variograms of realizations
    {avgfl}        -output file for average variogram
    {griddef}
     1                  -number of realizations
    2  25                 -number of directions, number of lags
     3  -1  0              -ixd(1),iyd(1),izd(1)
     0  0  1              -ixd(2),iyd(2),izd(2)
    1                     -standardize sill? (0=no, 1=yes)
    {nvario}                     -number of variograms
    {variotypes}
"""
vartemp = """{}   {}   {}             -tail variable, head variable, variogram type"""

In [ ]:
results = {}
for name, title in gg.log_progress(names.items()):
    parallelpars = []
    for ivar, var in enumerate(variables): 
        for ireal in range(nreal):
            fmtpar = varsimpar.format(
                datfl=f'{finaldir}{name}/Final/mergedbtr_{ireal}.gsb',
                nvar=1, varcols=ivar + 1, 
                ltrim=-998, 
                outfl=f'{finaldir}{name}/Final/varsim_{var}_{ireal}.out',
                avgfl=f'{finaldir}{name}/Final/varsimavg_{var}_{ireal}.out',
                griddef=griddef,
                nvario=1, variotypes=vartemp.format(1, 1, 1)
            )
            parallelpars.append(
                dict(parstr=fmtpar, 
                     parfile=f'{finaldir}{name}/Final/varsim_{var}_{ireal}.par')
            )
    gs.runparallel(varsim, parallelpars, processes=nprocesses, 
                   mute=True, reportprogress=True)
    for ivar, var in enumerate(variables): 
        results[name, var] = {'major': {}, 'vert': {}}
        for ireal in range(nreal):
            varfl = f'{finaldir}{name}/Final/varsim_{var}_{ireal}.out'
            vardat = gs.readfile(varfl)
            vardat.columns = ['idx', 'h', 'numpairs', 'vario',
                              'Variogram Number', 'azm', 'Calculation Dip',
                              'Variogram Type']
            avgfl = f'{finaldir}{name}/Final/varsimavg_{var}_{ireal}.out'
            parfile = f'{finaldir}{name}/Final/varsim_{var}_{ireal}.par'
            gs.cleantemp([varfl, avgfl, parfile])
            results[name, var]['major'][ireal] = \
                vardat.loc[vardat['idx'] == 1].reset_index(drop=True)
            results[name, var]['vert'][ireal] = \
                vardat.loc[vardat['idx'] == 2].reset_index(drop=True)

In [ ]:
rm.PYGSDEFAULTS['units'] = 'm'
rm.PYGSDEFAULTS['unit'] = 'm' 
qf = gg.QuickFormatter(fs=6)
def plotvarsim(varsimdict, refmodel, title, axs=None):
    if axs is None: 
        axs = qf.getax(2)
    ax = axs[0]
    for ireal in range(nreal):
        ax = gs.varplt(varsimdict['major'][ireal], experimental=False, ax=ax,
                       color='grey', lw=0.25, label='_nolegend_')
    maxh = varsimdict['major'][ireal]['h'].max()
    ax = gs.varplt(refmodel.principlepts('major', maxh), experimental=False, ax=ax,
                   color='red', lw=1, zorder=5, label='Global Model')
    ax.plot(np.nan, color='grey', lw=0.25, label='Realizations')
    ax.set_xbound(0)
    qf(ax, annot=f'{title}\nHorizontal', 
       annxy=(0.02, 1.01), annha='left', annva='top', annfs=6)
    ax.legend(loc='lower right', fontsize='x-small')
    ax.set_ylabel('$\\gamma$', fontsize=10)
    ax = axs[1]
    for ireal in range(nreal):
        ax = gs.varplt(varsimdict['vert'][ireal], experimental=False, ax=ax,
                       color='grey', lw=0.25, label='_nolegend_')
    maxh = varsimdict['vert'][ireal]['h'].max()
    ax = gs.varplt(refmodel.principlepts('vert', maxh), experimental=False, ax=ax,
                   color='red', lw=0.5, zorder=5, label='Global Model')
    ax.plot(np.nan, color='grey', lw=0.25, label='Realizations')
    ax.set_xbound(0)
    qf(ax, annot=f'Vertical', 
       annxy=(0.02, 1.01), annha='left', annva='top', annfs=6)
    ax.legend(loc='lower right', fontsize='x-small')
    ax.set_ylabel('$\\gamma$', fontsize=10)


In [ ]:
ouvarmodels = gg.load_pickle('oilsands_ouvarmodels.pkl')

In [ ]:
from gglib.utils import traverse

In [ ]:
topnames = list(names.keys())[:4]
for var in variables: 
    nplot = len(topnames)
    outeraxes = qf.nestedax((2, nplot), order='row', figsize=(6 * (nplot / 4), 3))
    for iname, (name, axs) in enumerate(zip(topnames, outeraxes)):
        refvario = gg.VarModel(ouvarmodels['nodomains', 1, var])
        simvarios = results[name, var]
        axs = plotvarsim(simvarios, refvario, names[name], axs)   
        qf(axs[0], fignum=iname, lblxy=(0.01, 1.06), lblha='right')
    qf(outeraxes, fs=5)
    for ax in traverse(outeraxes):
        ax.set_ylabel('$\\gamma$', fontsize=12)
        ax.yaxis.labelpad = 4.5
    chapterexport(f'oilsands_varsim_{var}_top.pdf')
    plt.show()

botnames = list(names.keys())[4:]
for var in variables: 
    nplot = len(botnames)
    outeraxes = qf.nestedax((2, len(botnames)), order='row', figsize=(6 * (nplot / 4), 3))
    for iname, (name, axs) in enumerate(zip(botnames, outeraxes)):
        refvario = gg.VarModel(ouvarmodels['nodomains', 1, var])
        simvarios = results[name, var]
        axs = plotvarsim(simvarios, refvario, names[name], axs)   
        qf(axs[0], fignum=iname + 4, lblxy=(0.01, 1.06), lblha='right')
    qf(outeraxes, fs=5)
    for ax in traverse(outeraxes):
        ax.set_ylabel('$\\gamma$', fontsize=12)
        ax.yaxis.labelpad = 4.5
    chapterexport(f'oilsands_varsim_{var}_bot.pdf')
    plt.show()